Take data from SQL Server to BigQuery

In [ ]:
import os
from google.cloud import bigquery
import pyodbc
import pandas as pd
import pandas_gbq

#Connect and export to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=14.225.9.147;'
                      'Database=Calie;'
                      'UID=Rossie;'
                      'PWD=Rossie2022@ReportMulti!@#$$;')
cursor = conn.cursor()

sql_query = "SELECT * FROM Ad_Database.dbo.Tiktok_AdReport"
df = pd.read_sql(sql_query, conn)

conn.close()


#Connect and import data to BigQuery
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'report-realtime-350003-c4cc0f514e7e.json'
client = bigquery.Client()

table_id = 'report-realtime-350003.Rossie.Tiktok_AdReport'

pandas_gbq.to_gbq(df, table_id, if_exists='replace')

In [2]:
import requests
import pandas as pd
from datetime import  datetime, timedelta
import os
from google.cloud import bigquery


def main_requests(page):
    time = int((datetime.now()-timedelta(days=7)).timestamp())
    url = "https://pos.pages.fm/api/v1/shops/5118364/orders?api_key=ba7023239ae44f8db8c38591fdb5cf1a&updateStatus=inserted_at&page_size=100"
    response = requests.request("GET", url + f'&page_number={page}' + f'&startDateTime={time}')
    return response.json()

def deleted_order_requests():
    time = int((datetime.now()-timedelta(days=7)).timestamp())
    url = "https://pos.pages.fm/api/v1/shops/5118364/orders?api_key=ba7023239ae44f8db8c38591fdb5cf1a&updateStatus=inserted_at&page_size=200&status=7"
    response = requests.request("GET", url + f'&startDateTime={time}')
    return response.json()

def get_pages(response):
    return response['total_pages']

def exact(list):
    result = []
    for i in list:
        if i is not None:
            result.append(i)
        else: pass
    if len(result) == 0:
        return None
    else: return result[0]

def exact_date(date):
    if date is None:
        return None
    else:
        return (datetime.strptime(date,  '%Y-%m-%dT%H:%M:%S') + timedelta(hours=7)).isoformat()

demand_list = ['IB','xng','xn2dt','xn1dt','xndt','xns','xn3','xn2','xn1','xn','cb2','cb1','call','tr','cb','ut']
cskh_list = ['HOTLINE','KH Mới','KQL2','KQL1','KQL']
failed_reason_list = ['Không tiếp cận', 'Giá', 'Sản phẩm']

def get_orders(response):
    orders_list = []

    for item in response['data']:
        id = str(item['id'])     
        char =      str(item['id']),\
                    (datetime.strptime(item['updated_at'], '%Y-%m-%dT%H:%M:%S.%f') + timedelta(hours=7)).isoformat(),\
                    (datetime.strptime([x['updated_at'] for x in item['status_history'] if x['status']==0][0], '%Y-%m-%dT%H:%M:%S') + timedelta(hours=7)).isoformat(),\
                    exact([x.get('name') if x['status']==0 else None for x in item['status_history']]),\
                    item['status'],\
                    item['order_sources_name'],\
                    str(item['page_id']) if item['page_id'] is not None else None,\
                    item['bill_full_name'],\
                    item['bill_phone_number'],\
                    item['total_price'],\
                    item['shipping_fee'],\
                    item['ad_id'],\
                    item['shipping_address']['province_name'],\
                    exact([x.get('name') if x['status']==1 else None for x in item['status_history']]),\
                    item['partner']['partner_name'] if item['partner'] is not None else None,\
                    item['partner']['extend_code'] if item['partner'] is not None else None,\
                    exact_date(exact([x['updated_at'] if x['status']==1 else None for x in item['status_history']])) ,\
                    exact_date(exact([x['updated_at'] if x['status']==2 else None for x in item['status_history']])) ,\
                    item['shipping_address']['full_address'] if item['shipping_address'] is not None else None,\
                    item['shipping_address']['full_name'] if item['shipping_address'] is not None else None,\
                    item['partner_fee'],\
                    item['total_discount'],\
                    ', '.join(item['customer_needs']) if item.get('customer_needs') else None,\
                    item['account_name'],\
                    exact_date(item['time_assign_seller']),\
                    item['assigning_seller'].get('name') if item['assigning_seller'] is not None else None,\
                    exact_date(item['time_assign_care']),\
                    item['marketer'].get('name') if item['marketer'] is not None else None,\
                    ', '.join(x.get('name') for x in item['tags']) if item['tags'] != [] else None,\
                    exact([x.get('name') if x.get('name') in demand_list else None for x in item.get('tags')]),\
                    exact([x.get('name') if x.get('name') in cskh_list else None for x in item.get('tags')]),\
                    exact([x.get('name') if x.get('name') in failed_reason_list else None for x in item.get('tags')]),\
                    item['customer']['date_of_birth'],\
                    item['cod']

        char = [None if (val is None or val == '') else val for val in char]
        orders_list.append(char)
    
    return orders_list

def get_orders_detail(response):
    orders_detail_list = []

    for item in response['data']:
        id = str(item['id']) 
        for i in item['items']: 
            row = str(i['id']),\
                id,\
                i['variation_info'].get('name'),\
                i['variation_info']['product_display_id'],\
                i['quantity'],\
                i['variation_info']['retail_price'],\
                i['discount_each_product']        
            row = [None if (val is None or val == '') else val for val in row]
            orders_detail_list.append(row)
    
    return orders_detail_list

orders_df = []
#order_detail_df = []

for i in range(1,get_pages(main_requests(1))+1):
    orders_df.extend(get_orders(main_requests(i)))
    #order_detail_df.extend(get_orders_detail(main_requests(i)))

orders_df.extend(get_orders(deleted_order_requests()))
#order_detail_df.extend(get_orders_detail(deleted_order_requests()))

orders_df = pd.DataFrame(orders_df)
#order_detail_df = pd.DataFrame(order_detail_df)

orders_df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,3322424,2024-01-11T13:01:33.572740,2024-01-11T13:00:33,None,0,FB,2105217262848989,Lê Vẹn,0788936473,0,...,None,None,None,None,tv,None,None,None,None,0
1,3322423,2024-01-11T12:17:35.000826,2024-01-11T12:12:57,None,1,FB,106200578847723,LÂM THỊ THANH NGA,0905931413,199000,...,2024-01-11T12:17:35,Nhàn Trần,None,None,None,None,None,None,None,219000
2,3322422,2024-01-11T12:26:57.902320,2024-01-11T12:07:15,None,1,FB,390723471777993,Han Hanh,0339789257,390000,...,2024-01-11T12:26:57,Nguyễn Thanh Nhàn,None,None,tv,None,None,None,None,195000
3,3322421,2024-01-11T11:11:04.496370,2024-01-11T11:07:28,None,1,FB,106200578847723,Hiền Đỗ,0936442536,199000,...,2024-01-11T11:11:04,Nhàn Trần,None,None,None,None,None,None,None,219000
4,3322420,2024-01-11T11:24:02.052054,2024-01-11T10:53:11,None,1,FB,390723471777993,Thắm Lê,0975533622,119000,...,2024-01-11T11:24:01,Nhàn Trần,None,None,tv,None,None,None,None,119000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,3322158,2024-01-05T15:26:54.710921,2024-01-04T14:33:37,None,3,FB,390723471777993,Ly Thảo Nguyễn,0908350567,119000,...,2024-01-04T17:39:57,Nhàn Trần,None,None,"tv, Chưa đối soát",None,None,None,None,119000
264,3322157,2024-01-09T10:58:44.122965,2024-01-04T14:09:27,None,3,FB,390723471777993,Thuy Pham,0988932814,119000,...,2024-01-08T09:46:13,Nguyễn Thanh Nhàn,None,None,Chưa đối soát,None,None,None,None,119000
265,3322156,2024-01-10T15:55:10.750338,2024-01-04T13:57:16,None,3,FB,106200578847723,Binbin Yeu,0342543552,199000,...,2024-01-04T13:58:44,Nguyễn Thanh Nhàn,None,None,Chưa đối soát,None,None,None,None,219000
266,3322155,2024-01-08T11:15:05.230034,2024-01-04T13:46:24,None,3,FB,390723471777993,Bich Mai Mai,0961040349,390000,...,2024-01-04T14:02:36,Nhàn Trần,None,None,"tv, Chưa đối soát",None,None,None,None,195000
